# Day 3: Kalman Filter for Sideslip Angle Estimation


## 🎯 Learning Goals
1. Understand why **sideslip angle estimation** is crucial in autonomous driving.  
2. Learn the basics of the **Kalman Filter**.  
3. Implement a simple Kalman Filter to estimate lateral velocity and sideslip angle.  
4. Compare true vs estimated values in a simulation.



## 1. Why Estimate Sideslip Angle?
- **Sideslip angle ($\beta$)** = difference between the vehicle’s direction of travel and its heading angle.

$$
\beta = \arctan\left(\frac{v_y}{v_x}\right)
$$

Where:
- $v_y$ = lateral velocity  
- $v_x$ = longitudinal velocity  

### Importance
- Determines if the vehicle is understeering or oversteering.  
- Critical for **stability control systems** (ESP, ADAS).  
- Not directly measurable with standard sensors → must be **estimated**.



## 2. Basics of Kalman Filter
The Kalman Filter provides optimal estimates of hidden states in noisy systems.

### State-Space Form
- **State:** $x_k = A x_{k-1} + B u_{k-1} + w_{k-1}$  
- **Measurement:** $z_k = H x_k + v_k$  

Where:
- $x_k$: state vector (e.g., $[v_y, r]$ lateral velocity & yaw rate)  
- $z_k$: measurement vector (e.g., yaw rate sensor, acceleration sensor)  
- $w_k$: process noise  
- $v_k$: measurement noise  

### Kalman Filter Steps
1. **Prediction:**
   - $\hat{x}^-_k = A \hat{x}_{k-1} + B u_{k-1}$  
   - $P^-_k = A P_{k-1} A^T + Q$
2. **Update:**
   - $K_k = P^-_k H^T (H P^-_k H^T + R)^{-1}$  
   - $\hat{x}_k = \hat{x}^-_k + K_k (z_k - H \hat{x}^-_k)$  
   - $P_k = (I - K_k H) P^-_k$


## 3. Python Simulation: Kalman Filter for Sideslip Angle

In [ ]:

import numpy as np
import matplotlib.pyplot as plt

# Simulation parameters
dt = 0.1
T = 20
time = np.arange(0, T, dt)
N = len(time)

# Vehicle parameters
vx = 15.0  # constant longitudinal velocity [m/s]
true_vy = np.sin(0.2*time)  # simulate lateral velocity (wavy motion)
true_r = 0.05*np.cos(0.2*time)  # simulate yaw rate

# True sideslip angle
true_beta = np.arctan2(true_vy, vx)

# Measurements with noise
meas_ay = np.gradient(true_vy, dt) + np.random.normal(0, 0.05, N)
meas_r = true_r + np.random.normal(0, 0.01, N)

# Kalman Filter initialization
A = np.array([[1, dt],
              [0, 1]])

H = np.eye(2)
Q = np.array([[0.001, 0],
              [0, 0.001]])
R = np.array([[0.05, 0],
              [0, 0.01]])

x_est = np.array([0, 0])  # [vy, r]
P = np.eye(2)

vy_est, r_est, beta_est = [], [], []

for k in range(N):
    # Prediction
    x_pred = A @ x_est
    P_pred = A @ P @ A.T + Q

    # Measurement vector
    z = np.array([meas_ay[k]*dt, meas_r[k]])

    # Update
    K = P_pred @ H.T @ np.linalg.inv(H @ P_pred @ H.T + R)
    x_est = x_pred + K @ (z - H @ x_pred)
    P = (np.eye(2) - K @ H) @ P_pred

    vy_est.append(x_est[0])
    r_est.append(x_est[1])
    beta_est.append(np.arctan2(x_est[0], vx))

# Plot results
plt.figure(figsize=(12,5))

plt.subplot(1,2,1)
plt.plot(time, true_beta, label="True β")
plt.plot(time, beta_est, label="Estimated β")
plt.xlabel("Time (s)")
plt.ylabel("Sideslip Angle (rad)")
plt.title("True vs Estimated Sideslip Angle")
plt.legend()
plt.grid(True)

plt.subplot(1,2,2)
plt.plot(time, true_vy, label="True vy")
plt.plot(time, vy_est, label="Estimated vy")
plt.xlabel("Time (s)")
plt.ylabel("Lateral Velocity (m/s)")
plt.title("True vs Estimated Lateral Velocity")
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()



## 4. Observations
- The Kalman Filter provides a smooth estimate of **sideslip angle** ($\beta$).  
- Despite noisy measurements, estimation tracks the true values closely.  
- Estimation improves stability in autonomous vehicle control.

### ✅ Conclusion
- Sideslip angle estimation is essential for stability control.  
- Kalman Filters are powerful for fusing noisy sensor data.  
- This sets the stage for **Day 4: integrating KF into dual-axle PID control**.
